In [1]:
#re-imports modules that have changed
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [9]:
import os
import logging
import json
import datetime as dt
import pandas as pd
from src.candidate_fetcher import VoterInfo
from src.utils import CloudStorageClient


In [ ]:
# Constants

In [4]:
TODAY = dt.datetime.now()

In [5]:
path = "/tmp/"
if not os.path.exists(path):
    os.mkdir(path)

In [44]:
# Load list of current elections 
bucket_name = "current_elections"
blob_name = 'current_elections.json'
filepath = path + blob_name
client = CloudStorageClient()
 
#load temp file to gcp  
client.download_file(filepath, bucket_name, blob_name=blob_name)

#load election json
with open(filepath) as json_file:
    data = json.load(json_file)

In [45]:
try: 
    elections = data['elections']
except KeyError as error: 
    logging.error(f"There are no current elections stored in file: 'gs://' {bucket_name} + '/' + {blob_name}")
    raise
except Exception as error: 
    logging.error(f"Error loading current elections from file: 'gs://' {bucket_name} + '/' + {blob_name}")
    logging.error(error)
    raise

In [46]:
bucket_name = 'address_locales'
blob_name = 'addresses_county.csv'
filepath = path + blob_name

#load temp file to gcp  
client.download_file(filepath, bucket_name, blob_name=blob_name)

#load address data
with open(filepath, 'rb') as file:
    data = pd.read_csv(filepath, encoding='utf-8')

In [54]:
try: 
    #process data - ensure necessary columns available
    addresses = data['address']
    state_abbr = data['state_abbr']
    data['fips'] = data['fips'].astype('str').str.zfill(5)
except KeyError as error: 
    logging.error(f"There are no current elections stored in file: 'gs://' {bucket_name} + '/' + {blob_name}")
    raise
except Exception as error: 
    logging.error(f"Error loading current elections from file: 'gs://' {bucket_name} + '/' + {blob_name}")
    logging.error(error)
    raise

In [ ]:
candidate_data = {}
for election in elections:
    civic = VoterInfo()
    results = []
    election_id = election['id']
    election_name = election['name']
    # Get state abbr from OCDid
    election_ocdid = election['ocdDivisionId'].split("/")[-1].split(":")[-1].upper()
    election_date = election['electionDay']
        
    # Except test election 
    if election_name == 'VIP Test Election': 
        continue
    elif election_ocdid == 'US': 
        print("True")
    else: 
        logging.info(f"Retrieving election data for {election_id}:{election_ocdid}")
        active = data.loc[data['state_abbr'] == election_ocdid, :]
        for index, row in active.iterrows():
            address = row['address']
            result = civic.fetch_voter_info(address, election_id)
            results.append(result)
    candidate_data[election_id] = results
        

In [57]:
candidate_data

,state_abbr,fips,name,lat,long,addresses,address,result
2311,RI,44001,Bristol County,41.705271,-71.285053,"['25 Patricia Ann Dr, Bristol, RI 02809, USA',...","25 Patricia Ann Dr, Bristol, RI 02809, USA","{'kind': 'civicinfo#voterInfoResponse', 'norma..."
2312,RI,44003,Kent County,41.677750,-71.576314,"['1012 Tiogue Ave, Coventry, RI 02816, USA', '...","1012 Tiogue Ave, Coventry, RI 02816, USA","{'kind': 'civicinfo#voterInfoResponse', 'norma..."
2313,RI,44005,Newport County,41.502732,-71.284063,"['127 Johnny Cake Hill, Middletown, RI 02842, ...","127 Johnny Cake Hill, Middletown, RI 02842, USA","{'kind': 'civicinfo#voterInfoResponse', 'norma..."
2314,RI,44007,Providence County,41.870488,-71.578242,"['66 W Greenville Rd, Greenville, RI 02828, US...","66 W Greenville Rd, Greenville, RI 02828, USA","{'kind': 'civicinfo#voterInfoResponse', 'norma..."
2315,RI,44009,Washington County,41.401162,-71.617612,"['79 Cherokee Bend, Charlestown, RI 02813, USA...","79 Cherokee Bend, Charlestown, RI 02813, USA","{'kind': 'civicinfo#voterInfoResponse', 'norma..."


In [ ]:
END HERE

In [ ]:
path = "/tmp/"
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
filepath = path + 'data.json'
if data: 
    #store as temp file
    try: 
        with open(filepath, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        logging.info(f"Data saved to '{filepath}''")
    except Exception as e: 
        logging.error(e)

In [ ]:
client = CloudStorageClient()
bucket_name = "current_elections"
 #load temp file to gcp as "current_elections.json"
client.upload_file(filepath, bucket_name, blob_name='current_elections.json')
 #load temp file to gcp by date
client.upload_file(filepath, bucket_name, blob_name=f'{TODAY}.json')

In [3]:
# Test main
from main import run_current_elections
run_current_elections()

{'kind': 'civicinfo#electionsQueryResponse', 'elections': [{'id': '2000', 'name': 'VIP Test Election', 'electionDay': '2021-06-06', 'ocdDivisionId': 'ocd-division/country:us'}, {'id': '4896', 'name': 'Wisconsin State Primary Election', 'electionDay': '2020-02-18', 'ocdDivisionId': 'ocd-division/country:us/state:wi'}, {'id': '4897', 'name': 'South Carolina Presidential Primary', 'electionDay': '2020-02-29', 'ocdDivisionId': 'ocd-division/country:us/state:sc'}, {'id': '4898', 'name': 'Alabama Presidential and State Primary Election', 'electionDay': '2020-03-03', 'ocdDivisionId': 'ocd-division/country:us/state:al'}, {'id': '4899', 'name': 'Arkansas Presidential Primary Election', 'electionDay': '2020-03-03', 'ocdDivisionId': 'ocd-division/country:us/state:ar'}, {'id': '4900', 'name': 'California Presidential Primary Election', 'electionDay': '2020-03-03', 'ocdDivisionId': 'ocd-division/country:us/state:ca'}, {'id': '4901', 'name': 'Colorado Presidential Primary Election', 'electionDay': '